In [117]:
import requests
import json
import numpy as np
from bs4 import BeautifulSoup

In [37]:
def lastfm_get(payload):

    headers = {'user-agent': 'naviak'}
    url = 'https://ws.audioscrobbler.com/2.0/'
    payload['api_key'] = 'd0cf7db68a5c604b1e2042a7d7ad1c64'
    payload['format'] = 'json'

    response = requests.get(url, headers=headers, params=payload)
    return response
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [123]:
r = lastfm_get({
    'method': 'chart.getTopTracks'
})

js = {key: r.json()['tracks']['track'][1][key] for key in ('artist','name','url')}
js['artist'] = js['artist']['name']
jprint(js)

{
    "artist": "Olivia Rodrigo",
    "name": "good 4 u",
    "url": "https://www.last.fm/music/Olivia+Rodrigo/_/good+4+u"
}


In [124]:
import time
from IPython.core.display import clear_output

responses = []

page = 1
total_pages = 90

while page <= total_pages:
    payload = {
        'method': 'chart.getTopTracks',
        'limit': 50,
        'page': page
    }

    print("Requesting page {}/{}".format(page, total_pages))
    clear_output(wait = True)

    response = lastfm_get(payload)

    if response.status_code != 200:
        print(response.text)
        break

    page = int(response.json()['tracks']['@attr']['page'])
    #total_pages = int(response.json()['tracks']['@attr']['totalPages'])

    responses.append(response)

    if not getattr(response, 'from_cache', False):
        time.sleep(0.1)

    # increment the page number
    page += 1

Requesting page 90/90


In [86]:
def prettify(response):
    response_ = response.json()
    per_page = int(response_['tracks']['@attr']['perPage'])
    response_prettifed = list()
    for i in range(per_page):
        rsp = {key: response_['tracks']['track'][i][key] for key in ('artist','name','url')}
        rsp['artist'] = rsp['artist']['name']
        response_prettifed.append(rsp)
    return response_prettifed

In [169]:
import pandas as pd
frames = [pd.DataFrame(prettify(r)) for r in responses]
tracks = pd.concat(frames, ignore_index=True)
tracks

,artist,name,url
0,Mariah Carey,All I Want for Christmas Is You,https://www.last.fm/music/Mariah+Carey/_/All+I...
1,Olivia Rodrigo,good 4 u,https://www.last.fm/music/Olivia+Rodrigo/_/goo...
2,Doja Cat,Kiss Me More (feat. SZA),https://www.last.fm/music/Doja+Cat/_/Kiss+Me+M...
3,Wham!,Last Christmas,https://www.last.fm/music/Wham%21/_/Last+Chris...
4,Lil Nas X,INDUSTRY BABY (feat. Jack Harlow),https://www.last.fm/music/Lil+Nas+X/_/INDUSTRY...
...,...,...,...
4495,Travis Scott,3500 (feat. Future & 2 Chainz),https://www.last.fm/music/Travis+Scott/_/3500+...
4496,Bruno Mars,Versace on the Floor,https://www.last.fm/music/Bruno+Mars/_/Versace...
4497,Kanye West,Never Let Me Down,https://www.last.fm/music/Kanye+West/_/Never+L...
4498,Lil Wayne,Love Me,https://www.last.fm/music/Lil+Wayne/_/Love+Me


In [143]:
def get_genres_and_yt(links):
    max_count = links.shape[0]
    genres_list = list()
    yt_links_list = list()
    count = 0
    for link in links:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, "html.parser")
        
        print("Getting genres in link {}/{}".format(count, max_count))
        genre_elements = soup.find_all("li", class_="tag")
        if genre_elements:
            genres_list.append(np.array([x.find('a').contents[0] for x in genre_elements]))
        else:
            genres_list.append([])
        clear_output(wait = True)
        
        print("Getting yt_link in link {}/{}".format(count, max_count))
        yt_elements = soup.find_all("a", class_="header-new-playlink", href=True)
        if yt_elements:
            yt_links_list.append(yt_elements[0]['href'])
        else:
            yt_links_list.append('None')
        clear_output(wait = True)
        count += 1
        time.sleep(0.1)
    return np.array(genres_list,dtype=object), np.array(yt_links_list, dtype=str)

In [144]:
genres, yt_links = get_genres_and_yt(tracks.url.values)

Getting yt_link in link 4499/4500


In [145]:
print(yt_links)

['https://www.youtube.com/watch?v=yXQViqx6GMY'
 'https://www.youtube.com/watch?v=gNi_6U5Pm_o'
 'https://www.youtube.com/watch?v=0EVVKs6DQLo' ...
 'https://www.youtube.com/watch?v=p4NvOKy7GOU'
 'https://www.youtube.com/watch?v=KY44zvhWhp4'
 'https://www.youtube.com/watch?v=mjd5fSPjZLQ']


In [170]:
tracks_copy = tracks.copy()
tracks_copy['yt_links'] = yt_links

In [171]:
tracks_copy

,artist,name,url,yt_links
0,Mariah Carey,All I Want for Christmas Is You,https://www.last.fm/music/Mariah+Carey/_/All+I...,https://www.youtube.com/watch?v=yXQViqx6GMY
1,Olivia Rodrigo,good 4 u,https://www.last.fm/music/Olivia+Rodrigo/_/goo...,https://www.youtube.com/watch?v=gNi_6U5Pm_o
2,Doja Cat,Kiss Me More (feat. SZA),https://www.last.fm/music/Doja+Cat/_/Kiss+Me+M...,https://www.youtube.com/watch?v=0EVVKs6DQLo
3,Wham!,Last Christmas,https://www.last.fm/music/Wham%21/_/Last+Chris...,https://www.youtube.com/watch?v=E8gmARGvPlI
4,Lil Nas X,INDUSTRY BABY (feat. Jack Harlow),https://www.last.fm/music/Lil+Nas+X/_/INDUSTRY...,https://www.youtube.com/watch?v=UTHLKHL_whs
...,...,...,...,...
4495,Travis Scott,3500 (feat. Future & 2 Chainz),https://www.last.fm/music/Travis+Scott/_/3500+...,https://www.youtube.com/watch?v=3qNaoLgHU94
4496,Bruno Mars,Versace on the Floor,https://www.last.fm/music/Bruno+Mars/_/Versace...,https://www.youtube.com/watch?v=-FyjEnoIgTM
4497,Kanye West,Never Let Me Down,https://www.last.fm/music/Kanye+West/_/Never+L...,https://www.youtube.com/watch?v=p4NvOKy7GOU
4498,Lil Wayne,Love Me,https://www.last.fm/music/Lil+Wayne/_/Love+Me,https://www.youtube.com/watch?v=KY44zvhWhp4


In [173]:
genr = genres.copy()
genr_df = pd.DataFrame(genr.tolist())

In [183]:
joined = tracks_copy.join(genr_df)
joined.rename(columns={key:f'Genre{key}' for key in range(8)}, inplace=True)
joined

,artist,name,url,yt_links,Genre0,Genre1,Genre2,Genre3,Genre4,Genre5,Genre6,Genre7
0,Mariah Carey,All I Want for Christmas Is You,https://www.last.fm/music/Mariah+Carey/_/All+I...,https://www.youtube.com/watch?v=yXQViqx6GMY,christmas,pop,xmas,mariah carey,pop,rnb,female vocalists,None
1,Olivia Rodrigo,good 4 u,https://www.last.fm/music/Olivia+Rodrigo/_/goo...,https://www.youtube.com/watch?v=gNi_6U5Pm_o,pop rock,rock,pop punk,sour,pop,pop rock,indie pop,None
2,Doja Cat,Kiss Me More (feat. SZA),https://www.last.fm/music/Doja+Cat/_/Kiss+Me+M...,https://www.youtube.com/watch?v=0EVVKs6DQLo,planet her,pop,rnb,pop rap,2021,pop,rnb,rap
3,Wham!,Last Christmas,https://www.last.fm/music/Wham%21/_/Last+Chris...,https://www.youtube.com/watch?v=E8gmARGvPlI,christmas,80s,pop,xmas,pop,80s,british,None
4,Lil Nas X,INDUSTRY BABY (feat. Jack Harlow),https://www.last.fm/music/Lil+Nas+X/_/INDUSTRY...,https://www.youtube.com/watch?v=UTHLKHL_whs,rap,hip-hop,trap,pop rap,2021,hip-hop,rap,country rap
...,...,...,...,...,...,...,...,...,...,...,...,...
4495,Travis Scott,3500 (feat. Future & 2 Chainz),https://www.last.fm/music/Travis+Scott/_/3500+...,https://www.youtube.com/watch?v=3qNaoLgHU94,None,None,None,None,None,None,None,None
4496,Bruno Mars,Versace on the Floor,https://www.last.fm/music/Bruno+Mars/_/Versace...,https://www.youtube.com/watch?v=-FyjEnoIgTM,rnb,sexy,ballad,soul,pop,pop,rnb,male vocalists
4497,Kanye West,Never Let Me Down,https://www.last.fm/music/Kanye+West/_/Never+L...,https://www.youtube.com/watch?v=p4NvOKy7GOU,hip hop,rap,kanye west,hip-hop,hip-hop,rap,hip hop,None
4498,Lil Wayne,Love Me,https://www.last.fm/music/Lil+Wayne/_/Love+Me,https://www.youtube.com/watch?v=KY44zvhWhp4,None,None,None,None,None,None,None,None


In [184]:
joined.to_csv('dataset.csv')

In [ ]:
df